# HAWK: Firma Digital Post-Cuántica basada en Retículos
## Implementación en Python

Este notebook muestra cómo funciona el esquema de firma digital HAWK.
HAWK es un algoritmo de firma post-cuántico cuya seguridad se basa en
problemas de retículos (lattices), lo que lo hace resistente frente a
ataques con ordenadores cuánticos.

Repositorio original (Licencia MIT): https://github.com/hawk-sign/hawk-py  
Autor del notebook: Cristian Yahir Nina Orellana


## Estructura del Notebook

1. Importación de librerías y definición de constantes
2. Componentes criptográficos principales
   - Raíces de la unidad
   - Operaciones con polinomios
   - Generación de aleatoriedad
   - Herramientas NTRU
   - Codificación y decodificación
   - Recuperación de los polinomios f y g
   - Componentes adicionales para la firma
   - Componentes adicionales para la verificación
3. Algoritmos finales
   - Generación de claves (`Gen`)
   - Firma (`Sign`)
   - Verificación (`Ver`)


## Importación de librerías y Constantes
Solo dar al play

In [ ]:
import numpy as np
import random
import hashlib
import math
from sympy.ntheory import primitive_root

In [ ]:
class HAWKParams:
    """
    Parámetros del esquema de firma digital post-cuántico HAWK.
    Soporta variantes de seguridad 256, 512 y 1024 bits (logn = 8, 9, 10).
    """

    # Tablas CDT (Cumulative Distribution Table) para el muestreo gaussiano
    HAWK256_T0 = [
        0x26B871FBD58485D45050, 0x07C054114F1DC2FA7AC9, 0x00A242F74ADDA0B5AE61,
        0x0005252E2152AB5D758B, 0x00000FDE62196C1718FC, 0x000000127325DDF8CEBA,
        0x0000000008100822C548, 0x00000000000152A6E9AE, 0x0000000000000014DA4A,
        0x0000000000000000007B,
    ]

    HAWK256_T1 = [
        0x13459408A4B181C718B1, 0x027D614569CC54722DC9, 0x0020951C5CDCBAFF49A3,
        0x0000A3460C30AC398322, 0x000001355A8330C44097, 0x00000000DC8DE401FD12,
        0x00000000003B0FFB28F0, 0x00000000000005EFCD99, 0x00000000000000003953,
        0x00000000000000000000,
    ]

    HAWK512_T0 = [
        0x2C058C27920A04F8F267, 0x0E9A1C4FF17C204AA058, 0x02DBDE63263BE0098FFD,
        0x005156AEDFB0876A3BD8, 0x0005061E21D588CC61CC, 0x00002BA568D92EEC18E7,
        0x000000CF0F8687D3B009, 0x0000000216A0C344EB45, 0x0000000002EDF0B98A84,
        0x0000000000023AF3B2E7, 0x00000000000000EBCC6A, 0x000000000000000034CF,
        0x00000000000000000006,
    ]

    HAWK512_T1 = [
        0x1AFCBC689D9213449DC9, 0x06EBFB908C81FCE3524F, 0x01064EBEFD8FF4F07378,
        0x0015C628BC6B23887196, 0x0000FF769211F07B326F, 0x00000668F461693DFF8F,
        0x0000001670DB65964485, 0x000000002AB6E11C2552, 0x00000000002C253C7E81,
        0x00000000000018C14ABF, 0x0000000000000007876E, 0x0000000000000000013D,
        0x00000000000000000000,
    ]

    HAWK1024_T0 = [
        0x2C583AAA2EB76504E560, 0x0F1D70E1C03E49BB683E, 0x031955CDA662EF2D1C48,
        0x005E31E874B355421BB7, 0x000657C0676C029895A7, 0x00003D4D67696E51F820,
        0x0000014A1A8A93F20738, 0x00000003DAF47E8DFB21, 0x0000000006634617B3FF,
        0x000000000005DBEFB646, 0x00000000000002F93038, 0x0000000000000000D5A7,
        0x00000000000000000021,
    ]

    HAWK1024_T1 = [
        0x1B7F01AE2B17728DF2DE, 0x07506A00B82C69624C93, 0x01252685DB30348656A4,
        0x001A430192770E205503, 0x00015353BD4091AA96DB, 0x000009915A53D8667BEE,
        0x00000026670030160D5F, 0x00000000557CD1C5F797, 0x00000000006965E15B13,
        0x00000000000047E9AB38, 0x000000000000001B2445, 0x000000000000000005AA,
        0x00000000000000000000,
    ]

    # Diccionario por nivel de seguridad (logn)
    PARAMS = {
        8: {
            "lenpriv": 96, "lenpub": 450, "lensig": 249, "n": 256,
            "qs": 2**32, "sigmasign": 1.010, "sigmaverify": 1.042,
            "sigmakrsec": 1.042, "lensalt": 14, "lenkgseed": 16,
            "lenhpub": 16, "low00": 5, "high00": 9, "low01": 8, "high01": 11,
            "high11": 13, "highs0": 12, "lows1": 5, "highs1": 9,
            "beta0": 1 / 250, "T0": HAWK256_T0, "T1": HAWK256_T1
        },
        9: {
            "lenpriv": 184, "lenpub": 1024, "lensig": 555, "n": 512,
            "qs": 2**64, "sigmasign": 1.278, "sigmaverify": 1.425,
            "sigmakrsec": 1.425, "lensalt": 24, "lenkgseed": 24,
            "lenhpub": 32, "low00": 5, "high00": 9, "low01": 9, "high01": 12,
            "high11": 15, "highs0": 13, "lows1": 5, "highs1": 9,
            "beta0": 1 / 1000, "T0": HAWK512_T0, "T1": HAWK512_T1
        },
        10: {
            "lenpriv": 360, "lenpub": 2440, "lensig": 1221, "n": 1024,
            "qs": 2**64, "sigmasign": 1.299, "sigmaverify": 1.571,
            "sigmakrsec": 1.974, "lensalt": 40, "lenkgseed": 40,
            "lenhpub": 64, "low00": 6, "high00": 10, "low01": 10, "high01": 14,
            "high11": 17, "highs0": 14, "lows1": 6, "highs1": 10,
            "beta0": 1 / 3000, "T0": HAWK1024_T0, "T1": HAWK1024_T1
        }
    }

    @staticmethod
    def get(logn: int, param: str):
        """
        Devuelve el parámetro solicitado para el esquema HAWK con un cierto logn.

        Args:
            logn (int): log2 del grado (8, 9, 10 para HAWK-256, 512, 1024)
            param (str): clave del parámetro a consultar

        Returns:
            valor del parámetro (int, float o lista)
        """
        if logn not in HAWKParams.PARAMS:
            raise ValueError(f"logn = {logn} no está soportado (usa 8, 9 o 10).")
        if param not in HAWKParams.PARAMS[logn]:
            raise ValueError(f"Parámetro '{param}' no encontrado en HAWK-{2**logn}.")
        return HAWKParams.PARAMS[logn][param]

## Componentes

### Raíces
Solo dar al play

In [ ]:
!wget https://raw.githubusercontent.com/hawk-sign/hawk-py/main/ntrugen/fft_constants.py
from fft_constants import roots_dict

--2025-06-10 16:46:42--  https://raw.githubusercontent.com/hawk-sign/hawk-py/main/ntrugen/fft_constants.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93741 (92K) [text/plain]
Saving to: ‘fft_constants.py’

fft_constants.py    100%[===================>]  91.54K  --.-KB/s    in 0.009s  

2025-06-10 16:46:42 (9.70 MB/s) - ‘fft_constants.py’ saved [93741/93741]



### Operaciones entre polinomios

En esta sección se implementan las operaciones fundamentales entre polinomios,
que constituyen la base algebraica del esquema de firma HAWK. Para mayor velocidad de computación se usan la NTT y FFT.

In [ ]:
class NTT:
    """
    Implementación de la Transformada de Número Teórico (NTT) para polinomios.

    La NTT es una versión discreta y modular de la Transformada Rápida de Fourier (FFT),
    utilizada para multiplicar polinomios de forma eficiente dentro de un anillo finito Z_p[X]/(X^n + 1).
    Esta clase proporciona métodos para:
    - Calcular la NTT directa (transformar un polinomio al dominio NTT).
    - Calcular la NTT inversa (recuperar el polinomio original).
    - Precalcular las raíces de la unidad necesarias para las transformadas.
    - Aplicar la multiplicación de polinomios en el dominio transformado.
    """
    @staticmethod
    def brv(x: int, b: int = 32) -> int:
        """
        Esta función invierte el orden de los bits de un número entero `x` en su representación
        binaria, considerando un total de `b` bits.
        Args:
            x : int
            b : int, opcional (por defecto, 32).
        """
        r = 0
        for i in range(b):
            r |= ((x >> i) & 1) << (b - 1 - i)
        return r

    @staticmethod
    def compute_zetas(root_of_unity: int, p: int, log_deg: int)-> list[int]:
        """
        Calcula las potencias de la raíz de la unidad en el orden requerido para la NTT.
        Args:
        root_of_unity : int
        p : int
        log_deg : int
        """
        x = root_of_unity
        zetas = [None for _ in range(1 << log_deg)]
        for u in range(1 << log_deg):
            zetas[u] = pow(x, NTT.brv(u, log_deg), p)
        return zetas

    @staticmethod
    def ntt(f: list[int], p: int, zetas: list[int] = None) -> list[int]:
        """
        Aplica la Transformada de Número Teórico (NTT) a un polinomio.
        Args:
            f : list[int]
            Formato: coeficiente
            p : int
            zetas : list[int], opcional (por defecto None)
        """
        if zetas is None:
            zetas, _ = NTT.get_roots(p, len(f))
        F = [int(f) for f in f]
        deg = len(f)
        l = deg // 2
        k = 1
        while l > 0:
            for s in range(0, deg, 2 * l):
                zeta = zetas[k]
                k += 1
                for j in range(s, s + l):
                    t = (F[j + l] * zeta) % p
                    F[j + l] = (F[j] - t) % p
                    F[j] = (F[j] + t) % p
            l = l // 2
        return F

    @staticmethod
    def intt(f: list[int], p: int, zetas: list[int] = None) -> list[int]:
        """
        Aplica la transformada inversa de número teórico (INTT) a un polinomio.
        Args:
            f : list[int]
            Formato: NTT
            p : int
            zetas : list[int], opcional (por defecto None)
        """
        if zetas is None:
            _, zetas = NTT.get_roots(p, len(f))
        F = [int(f) for f in f]
        deg = len(f)
        l = 1
        k = deg - 1
        while l < deg:
            for s in reversed(range(0, deg, 2 * l)):
                zeta = zetas[k]
                k -= 1
                for j in range(s, s + l):
                    t = F[j]
                    F[j] = (t + F[j + l]) % p
                    F[j + l] = (t - F[j + l]) % p
                    F[j + l] = (F[j + l] * zeta) % p
            l = l * 2

        ideg = pow(deg, p - 2, p)
        F = [(f * ideg) % p for f in F]
        return F

    @staticmethod
    def nttadj(u: list[int], p: int) -> list[int]:
        """
        Calcula el adjunto de un polinomio en el dominio NTT.
        Args:
            u : list[int]
            Formato: NTT
            p : int
        """
        zetas, izetas = NTT.get_roots(p, len(u))
        ui = NTT.intt(u, p, izetas)
        return NTT.ntt(PolOp.adjoint(ui), p, zetas)

    @staticmethod
    def get_roots(p: int, n: int) -> tuple[list[int], list[int]]:
        """
        Calcula las raíces de la unidad y su inversa en el dominio NTT.
        Args:
            p : int
            n : int
        """
        logn = int(math.log2(n))
        g0 = primitive_root(p)
        b = (p - 1) // (2 * n)
        g0 = (g0**b) % p

        zetas = NTT.compute_zetas(g0, p, logn)
        izetas = [pow(z, p - 2, p) for z in zetas]
        return zetas, izetas

    @staticmethod
    def poly_mul_ntt(p1: list[int], p2: list[int], p: int) -> list[int]:
        """
        Multiplica dos polinomios usando la Transformada de Número Teórico (NTT).
        Args:
            p1 : list[int]
            Formato: coeficiente
            p2 : list[int]
            Formato: coeficiente
            p : int
        """
        n = len(p1)

        zetas, izetas = NTT.get_roots(p, n)
        p1ntt = NTT.ntt(p1, p, zetas)
        p2ntt = NTT.ntt(p2, p, zetas)
        rntt = [(a * b) % p for a, b in zip(p1ntt, p2ntt)]
        m = NTT.intt(rntt, p, izetas)
        for i in range(n):
            if m[i] > (p - 1) // 2:
                m[i] -= p
        return m
class PolOp:
    """
    Clase que agrupa operaciones básicas entre polinomios en el contexto de Z o Z_p.

    Esta clase proporciona métodos estáticos para realizar operaciones fundamentales
    con polinomios, como suma, resta, multiplicación (por el método clásico) y obtención
    del adjunto. Estas operaciones son utilizadas en diversas partes del esquema HAWK,
    especialmente en la implementación del producto NTRU y en el preprocesamiento de
    datos para la firma.

    Métodos disponibles:
    - poly_mul_schoolbook: multiplicación clásica de polinomios con reducción modular.
    - poly_add: suma de dos polinomios.
    - poly_sub: resta de dos polinomios.
    - adjoint: devuelve el adjunto de un polinomio (inversión de coeficientes).
    """
    @staticmethod
    def poly_mul_schoolbook(p1: list[int], p2: list[int], p: int) -> list[int]:
        """
        Multiplica dos polinomios utilizando el algoritmo clásico,
        con reducción módulo X^n + 1 y coeficientes en ℤ_p.
        Args:
            p1 : list[int]
            Formato: coeficiente
            p2 : list[int]
            Formato: coeficiente
            p : int
        """
        deg = len(p1)
        r = [0 for _ in range(deg)]
        for i in range(deg):
            for j in range(deg):
                ij = i + j
                if ij >= deg:
                    r[ij % deg] -= ((p1[i] * p2[j])) % p
                else:
                    r[ij % deg] += ((p1[i] * p2[j])) % p

        r = [x % p for x in r]
        return r

    @staticmethod
    def adjoint(u: list[int]) -> list[int]:
        """
        Calcula el adjunto de un polinomio.
        Args:
            u : list[int]
            Formato: coeficiente
        """
        ustar = u.copy()
        n = len(u)
        for i in range(1, n):
            ustar[i] = -u[n - i]
        return ustar

    @staticmethod
    def poly_sub(p0: list[int], p1: list[int]) -> list[int]:
        """
        Resta dos polinomios.
        Args:
            p0 : list[int]
            Formato: coeficiente
            p1 : list[int]
            Formato: coeficiente
        """
        return [p0 - p1 for (p0, p1) in zip(p0, p1)]

    @staticmethod
    def poly_add(p0: list[int], p1: list[int]) -> list[int]:
        """
        Suma dos polinomios
        Args:
            p0 : list[int]
            Formato: coeficiente
            p1 : list[int]
            Formato: coeficiente
        """
        return [p0 + p1 for (p0, p1) in zip(p0, p1)]

In [ ]:
class PolProp:
    """
    Clase que agrupa propiedades y transformaciones útiles de polinomios.
    Métodos disponibles:
    - infnorm: calcula la norma infinito de un polinomio.
    - l2norm: calcula la norma cuadrática (euclídea) de un polinomio.
    - bytes_to_poly: convierte una secuencia de bytes en un polinomio binario.
    - isinvertible: determina si un polinomio es invertible módulo p.
    """
    @staticmethod
    def infnorm(poly: list[int]) -> int:
        """
        Calcula la norma infinito de un polinomio.
        Args:
            poly : list[int]
            Formato: coeficiente
        """
        return max([abs(p) for p in poly])

    @staticmethod
    def bytes_to_poly(h: bytes, n: int) -> list[int]:
        """
        Convierte una secuencia de bytes en un polinomio binario de grado n.
        Args:
            h : bytes
            n : int
        """
        h0 = [None] * n
        for i in range(n):
            h0[i] = (h[i // 8] >> (i % 8)) & 1
        return h0

    @staticmethod
    def l2norm(x: list[int]) -> int:
        """
        Calcula la norma euclídea al cuadrado de un polinomio.
        Args:
            x : list[int]
            Formato: coeficiente
        """
        x = [x * x for x in x]
        return sum(x)

    @staticmethod
    def isinvertible(poly: list[int], p: int) -> bool:
        """
        Verifica si un polinomio es invertible en Z_p[X]/(X^n + 1).
        Args:
            poly : list[int]
            Formato: coeficiente
            p : int
        """
        if p == 2:
            return (np.sum(poly) % 2) == 1

        polyntt = NTT.ntt(poly, p)
        return all([c != 0 for c in polyntt])


In [ ]:
class Common:
    q = 12 * 1024 + 1

    @staticmethod
    def split(f: list[int]) -> list[list[int]]:
        """
        Divide un polinomio en dos polinomios.
        Args:
            f: list[int]
            Formato: coeficiente
        """
        n = len(f)
        f0 = [f[2 * i + 0] for i in range(n // 2)]
        f1 = [f[2 * i + 1] for i in range(n // 2)]
        return [f0, f1]

    @staticmethod
    def merge(f_list: list[list[int]]) -> list[int]:
        """
        Fusiona dos polinomios en uno solo.
        Args:
            f_list: list[int]
            Format: coefficient
        """
        f0, f1 = f_list
        n = 2 * len(f0)
        f = [0] * n
        for i in range(n // 2):
            f[2 * i + 0] = f0[i]
            f[2 * i + 1] = f1[i]
        return f

    @staticmethod
    def sqnorm(v: list[int]) -> int:
        """
        Calcula la norma euclídea al cuadrado de un vector v.
        Args:
            v: list[int]
        """
        res = 0
        for elt in v:
            for coef in elt:
                res += coef**2
        return res


In [ ]:
class FFT:
    """
    Clase que implementa operaciones con polinomios en el dominio de la FFT.

    Esta clase proporciona funciones para calcular la transformada rápida de Fourier
    (FFT) y su inversa, así como operaciones polinómicas básicas (suma, resta,
    multiplicación, división, adjunto) tanto en el dominio de coeficientes como en
    el dominio FFT.

    Las funciones están orientadas a representar y manipular polinomios en R[x] o C[x].

    Métodos disponibles:
    - fft / ifft: transformada directa e inversa de un polinomio.
    - add / sub / mul / div: operaciones aritméticas en representación por coeficientes.
    - add_fft / sub_fft / mul_fft / div_fft: mismas operaciones en representación FFT.
    - adj / adj_fft: cálculo del adjunto en ambos dominios.
    - inv_fft: cálculo aproximado del inverso de un polinomio.
    - split_fft / merge_fft: funciones auxiliares para descomposición y combinación.
    """
    @staticmethod
    def split_fft(f_fft: list[complex]) -> list[list[complex]]:
        """
        Divide un polinomio en el dominio FFT en dos partes: f₀ y f₁.

        Esta función separa un polinomio transformado por FFT en sus componentes
        correspondientes a los coeficientes pares e impares originales, como parte
        del proceso recursivo de inversión o análisis de la FFT.

        Args:
            f_fft : list[complex]
            format : FFT.
        """
        n = len(f_fft)
        w = roots_dict[n]
        f0_fft = [0] * (n // 2)
        f1_fft = [0] * (n // 2)
        for i in range(n // 2):
            f0_fft[i] = 0.5 * (f_fft[2 * i] + f_fft[2 * i + 1])
            f1_fft[i] = 0.5 * (f_fft[2 * i] - f_fft[2 * i + 1]) * w[2 * i].conjugate()
        return [f0_fft, f1_fft]

    @staticmethod
    def merge_fft(f_list_fft: list[list[complex]]) -> list[complex]:
        """
        Fusiona dos polinomios FFT en uno solo.
        Args:
            f_list: list[list[complex]]
            Format: FFT
        """
        f0_fft, f1_fft = f_list_fft
        n = 2 * len(f0_fft)
        w = roots_dict[n]
        f_fft = [0] * n
        for i in range(n // 2):
            f_fft[2 * i + 0] = f0_fft[i] + w[2 * i] * f1_fft[i]
            f_fft[2 * i + 1] = f0_fft[i] - w[2 * i] * f1_fft[i]
        return f_fft

    @staticmethod
    def fft(f: list[float]) -> list[complex]:
        """
        Calcula la Transformada Rápida de Fourier (FFT) de un polinomio módulo (xⁿ + 1).
        Args:
            f : list[float]
            Formato: coeficiente
        """
        n = len(f)
        if n > 2:
            f0, f1 = Common.split(f)
            f0_fft = FFT.fft(f0)
            f1_fft = FFT.fft(f1)
            f_fft = FFT.merge_fft([f0_fft, f1_fft])
        elif n == 2:
            f_fft = [0] * n
            f_fft[0] = f[0] + 1j * f[1]
            f_fft[1] = f[0] - 1j * f[1]
        return f_fft

    @staticmethod
    def ifft(f_fft: list[complex]) -> list[float]:
        """
        Calcula la transformada inversa FFT de un polinomio módulo (xⁿ + 1).

        Args:
            f_fft : list[complex]
            Formato: FFT
        """
        n = len(f_fft)
        if n > 2:
            f0_fft, f1_fft = FFT.split_fft(f_fft)
            f0 = FFT.ifft(f0_fft)
            f1 = FFT.ifft(f1_fft)
            f = Common.merge([f0, f1])
        elif n == 2:
            f = [0] * n
            f[0] = f_fft[0].real
            f[1] = f_fft[0].imag
        return f

    @staticmethod
    def add(f: list[int], g: list[int]) -> list[int]:
        """
        Suma de dos polinomios en representación por coeficientes.
        Args:
            f : list[int]
            g : list[int]
            Formato: coeficiente
        """
        assert len(f) == len(g)
        deg = len(f)
        return [f[i] + g[i] for i in range(deg)]

    @staticmethod
    def neg(f: list[int]) -> list[int]:
        """
        Negación de un polinomio (cualquier representación).

        Args:
            f : list[int]
        """
        deg = len(f)
        return [-f[i] for i in range(deg)]

    @staticmethod
    def sub(f: list[int], g: list[int]) -> list[int]:
        """
        Resta de dos polinomios (cualquier representación).

        Args:
            f : list[int]
            g : list[int]
        """
        return FFT.add(f, FFT.neg(g))

    @staticmethod
    def mul(f: list[float], g: list[float]) -> list[float]:
        """
        Multiplicación de dos polinomios en representación por coeficientes.

        Args:
            f : list[float]
            g : list[float]
            Formato: coeficiente
        """
        return FFT.ifft(FFT.mul_fft(FFT.fft(f), FFT.fft(g)))

    @staticmethod
    def div(f: list[float], g: list[float]) -> list[float]:
        """
        División de dos polinomios en representación por coeficientes.

        Args:
            f : list[float]
            g : list[float]
            Formato: coeficiente
        """
        return FFT.ifft(FFT.div_fft(FFT.fft(f), FFT.fft(g)))

    @staticmethod
    def adj(f: list[int]) -> list[int]:
        """
        Adjunto de un polinomio en representación por coeficientes.

        Args:
            f : list[int]
            Formato: coeficiente
        """
        return FFT.ifft(FFT.adj_fft(FFT.fft(f)))

    @staticmethod
    def add_fft(f_fft: list[complex], g_fft: list[complex]) -> list[complex]:
        """
        Suma de dos polinomios en representación FFT.

        Args:
            f_fft : list[complex]
            g_fft : list[complex]
            Formato: FFT
        """
        return FFT.add(f_fft, g_fft)

    @staticmethod
    def sub_fft(f_fft: list[complex], g_fft: list[complex]) -> list[complex]:
        """
        Resta de dos polinomios en representación FFT.

        Args:
            f_fft : list[complex]
            g_fft : list[complex]
            Formato: FFT
        """
        return FFT.sub(f_fft, g_fft)

    @staticmethod
    def mul_fft(f_fft: list[complex], g_fft: list[complex]) -> list[complex]:
        """
        Multiplicación punto a punto de dos polinomios en representación FFT.

        Args:
            f_fft : list[complex]
            g_fft : list[complex]
            Formato: FFT
        """
        deg = len(f_fft)
        return [f_fft[i] * g_fft[i] for i in range(deg)]

    @staticmethod
    def div_fft(f_fft: list[complex], g_fft: list[complex]) -> list[complex]:
        """
        División punto a punto de dos polinomios en representación FFT.

        Args:
            f_fft : list[complex]
            g_fft : list[complex]
            Formato: FFT
        """
        assert len(f_fft) == len(g_fft)
        deg = len(f_fft)
        return [f_fft[i] / g_fft[i] for i in range(deg)]

    @staticmethod
    def inv_fft(poly: list[float]) -> list[float]:
        """
        Calcula una inversión aproximada de un polinomio en el dominio FFT.

        Args:
            poly : list[float]
            Formato: coeficiente
        """
        n = len(poly)
        fftpoly = FFT.fft(poly)
        for u in range(n // 2):
            fftpoly[u] = 1.0 / fftpoly[u].real
        for u in range(n // 2):
            fftpoly[u + n // 2] = 0.0

        return FFT.ifft(fftpoly)

    @staticmethod
    def adj_fft(f_fft: list[complex]) -> list[complex]:
        """
        Adjunto de un polinomio en representación FFT.

        Args:
            f_fft : list[complex]
                Polinomio en el dominio FFT.

        Returns:
            list[complex]
                Polinomio adjunto, obtenido conjugando los coeficientes en FFT.
        """
        deg = len(f_fft)
        return [f_fft[i].conjugate() for i in range(deg)]


    """This value is the ratio between:
        - The degree n
        - The number of complex coefficients of the NTT
    While here this ratio is 1, it is possible to develop a short NTT such that it is 2.
    """
    fft_ratio = 1


In [ ]:
class FFTfp:
    """
    Clase que implementa la FFT e iFFT en punto fijo para enteros de 32 bits.

    Esta clase proporciona una versión de la Transformada Rápida de Fourier (FFT)
    adaptada al trabajo en aritmética de punto fijo.

    Se utiliza principalmente en el preprocesamiento de firmas para convertir
    polinomios enteros a una representación FFT sin recurrir a números en
    coma flotante, mejorando así la eficiencia y compatibilidad con hardware limitado.

    Métodos disponibles:
    - delta: calcula las raíces de la unidad utilizadas en la FFT en punto fijo.
    - sgn: obtiene el bit de signo de un número.
    - fft: aplica la FFT en punto fijo a un polinomio.
    - invfft: aplica la FFT inversa en punto fijo y recupera el polinomio original.
    """
    @staticmethod
    def delta(k: int) -> tuple[np.int32, np.int32]:
        """
        Calcula una raíz de la unidad en formato de punto fijo (ver Sección 3.6).

        Args:
            k : int
        Returns:
            tuple[np.int32, np.int32]
                Parte real e imaginaria de la raíz de la unidad en formato punto fijo.
        """
        d = np.exp(2 * 1j * np.pi * NTT.brv(k, b=10) / 2048)
        d = d * (2**31)
        re = np.int32(np.round(d.real))
        im = np.int32(np.round(d.imag))
        return re, im

    @staticmethod
    def sgn(x: int) -> int:
        """
        Devuelve el bit de signo de un número entero.

        Args:
            x : int
                Número entero de entrada.

        Returns:
            int
                1 si x < 0, 0 en caso contrario.
        """
        if x < 0:
            return 1
        return 0

    @staticmethod
    def fft(a: list[int]) -> list[np.int32]:
        """
        Calcula la transformada rápida de Fourier en punto fijo (ver Algoritmo 16).ç
        Args:
            a : list[int]
            Formato: coeficiente

        Returns:
            list[np.int32]
                Representación en FFT del polinomio, en formato punto fijo.
        """
        n = len(a)
        afft = a.copy()
        t = n // 2
        m = 2
        while m < n:
            v0 = 0
            for u in range(0, m // 2):
                e_re, e_im = FFTfp.delta(u + m)
                e_re = np.int64(e_re)
                e_im = np.int64(e_im)

                for v in range(v0, v0 + t // 2):
                    x1_re = np.int64(afft[v])
                    x1_im = np.int64(afft[v + n // 2])

                    x2_re = np.int64(afft[v + t // 2])
                    x2_im = np.int64(afft[v + t // 2 + n // 2])

                    t_re = x2_re * e_re - x2_im * e_im
                    t_im = x2_re * e_im + x2_im * e_re

                    afft[v] = np.int32((2**31 * x1_re + t_re) // 2**32)
                    afft[v + n // 2] = np.int32((2**31 * x1_im + t_im) // 2**32)
                    afft[v + t // 2] = np.int32((2**31 * x1_re - t_re) // 2**32)
                    afft[v + t // 2 + n // 2] = np.int32(
                        (2**31 * x1_im - t_im) // 2**32
                    )
                v0 = v0 + t
            t = t // 2
            m = 2 * m

        return afft

    @staticmethod
    def invfft(afft: list[np.int32]) -> list[np.int32]:
        """
        Calcula la transformada rápida de Fourier inversa en punto fijo (ver Algoritmo 17).
        Args:
            afft : list[np.int32]
            Formato: FFTfp

        Returns:
            list[np.int32] polinomio original.
        """
        n = len(afft)
        a = afft.copy()
        t = 2
        m = n // 2
        while m > 1:
            v0 = 0
            for u in range(0, m // 2):
                eta_re, eta_im = FFTfp.delta(u + m)

                eta_re = np.int64(eta_re)
                eta_im = np.int64(-eta_im)

                for v in range(v0, v0 + t // 2):
                    x1_re = a[v]
                    x1_im = a[v + n // 2]

                    x2_re = a[v + t // 2]
                    x2_im = a[v + t // 2 + n // 2]

                    t1_re = x1_re + x2_re
                    t1_im = x1_im + x2_im
                    t2_re = x1_re - x2_re
                    t2_im = x1_im - x2_im

                    a[v] = t1_re // 2
                    a[v + n // 2] = t1_im // 2
                    a[v + t // 2] = np.int32(
                        (np.int64(t2_re) * eta_re - np.int64(t2_im) * eta_im) // 2**32
                    )
                    a[v + t // 2 + n // 2] = np.int32(
                        (np.int64(t2_re) * eta_im + np.int64(t2_im) * eta_re) // 2**32
                    )
                v0 = v0 + t
            t = 2 * t
            m = m // 2

        return a

### Generación de aleatoriedad

In [ ]:
class RngContext:
    """
    Clase para generar aleatoriedad de forma determinista usando SHAKE256.

    Equivale al contexto de generación de números aleatorios (rngcontext)
    de la implementación de referencia en C del esquema HAWK. Utiliza
    SHAKE256 como generador de flujo pseudoaleatorio a partir de una semilla.

    Métodos:
    - random(size): devuelve un array de bytes aleatorios del tamaño solicitado.
    """

    def __init__(self, seed):
        self.shake256 = hashlib.shake_256()
        self.shake256.update(seed.tobytes())
        self.i = 0

    def random(self, size):
        r = np.frombuffer(self.shake256.digest(self.i + size), dtype=np.uint8)
        r = r[self.i : self.i + size]
        self.i += size
        return r


def SHAKE256x4(message: bytes, num: int) -> list[int]:
    """
    Implementa SHAKE256x4 (ver Sección 3.3.2) para generar aleatoriedad en paralelo.

    Esta función genera 4 flujos SHAKE256 paralelos a partir de un mismo mensaje
    concatenado con los valores [0], [1], [2] y [3]. Luego entrelaza la salida
    de los cuatro para obtener una secuencia unificada de enteros de 64 bits.

    Args:
        message : bytes
            Mensaje de entrada común para las cuatro instancias de SHAKE256.
        num : int
            Número total de enteros de 64 bits que se desean como salida.

    Returns:
        list[int]
            Lista de `num` enteros de 64 bits generados en forma entrelazada
            a partir de las cuatro salidas SHAKE256.
    """
    shake256x4 = [None] * 4
    digest = [None] * 4
    for i in range(4):
        shake256x4[i] = hashlib.shake_256()
        shake256x4[i].update(message + bytes([i]))
        digest[i] = shake256x4[i].digest(
            int(num * 8 / 4)
        )  # *8 (8 bytes in 64 bits) / 4 (for the 4 streams)

    # Convert the four streams in the interleaved form
    j = 0
    y = [None] * int(num)
    for i in range(int(num / 4)):
        y[j + 0] = int.from_bytes(digest[0][i * 8 : (i + 1) * 8], byteorder="little")
        y[j + 1] = int.from_bytes(digest[1][i * 8 : (i + 1) * 8], byteorder="little")
        y[j + 2] = int.from_bytes(digest[2][i * 8 : (i + 1) * 8], byteorder="little")
        y[j + 3] = int.from_bytes(digest[3][i * 8 : (i + 1) * 8], byteorder="little")
        j = j + 4
    return y


### Herramientas NTRU

In [ ]:
q = 1
def karatsuba(a: list[float], b: list[float], n: int) -> list[float]:
    """
    Multiplicación de Karatsuba entre dos polinomios.
    Args:
        a : list[float]
        b : list[float]
        n : int
        Formato: coeficiente
    Returns:
        list[float]
            Polinomio resultado de la multiplicación (longitud 2n).
    """
    if n == 1:
        return [a[0] * b[0], 0]
    else:
        n2 = n // 2
        a0 = a[:n2]
        a1 = a[n2:]
        b0 = b[:n2]
        b1 = b[n2:]
        ax = [a0[i] + a1[i] for i in range(n2)]
        bx = [b0[i] + b1[i] for i in range(n2)]
        a0b0 = karatsuba(a0, b0, n2)
        a1b1 = karatsuba(a1, b1, n2)
        axbx = karatsuba(ax, bx, n2)
        for i in range(n):
            axbx[i] -= a0b0[i] + a1b1[i]
        ab = [0] * (2 * n)
        for i in range(n):
            ab[i] += a0b0[i]
            ab[i + n] += a1b1[i]
            ab[i + n2] += axbx[i]
        return ab


def karamul(a: list[float], b: list[float]) -> list[float]:
    """
    Multiplicación de Karatsuba seguida de reducción módulo (xⁿ + 1).

    Args:
        a : list[float]
        b : list[float]
        Formato: coeficiente
    Returns:
        list[float]
            Resultado reducido de la multiplicación.
    """
    n = len(a)
    ab = karatsuba(a, b, n)
    abr = [ab[i] - ab[i + n] for i in range(n)]
    return abr


def galois_conjugate(a: list[float]) -> list[float]:
    """
    Conjugado de Galois de un elemento en Q[x] / (xⁿ + 1).

    El conjugado de Galois de a(x) es simplemente a(-x).

    Args:
        a : list[float]
            Polinomio de entrada.

    Returns:
        list[float]
            Polinomio conjugado.
    """
    n = len(a)
    return [((-1) ** i) * a[i] for i in range(n)]


def field_norm(a: list[float]) -> list[float]:
    """
    Proyecta un polinomio de Q[x] / (xⁿ + 1) a Q[x] / (xⁿ/² + 1).

    Esta operación se conoce como "norma de campo" y es válida si n es potencia de 2.

    Args:
        a : list[float]
        Formato: coeficiente
    Returns:
        list[float]
            Polinomio proyectado.
    """
    n2 = len(a) // 2
    ae = [a[2 * i] for i in range(n2)]
    ao = [a[2 * i + 1] for i in range(n2)]
    ae_squared = karamul(ae, ae)
    ao_squared = karamul(ao, ao)
    res = ae_squared[:]
    for i in range(n2 - 1):
        res[i + 1] -= ao_squared[i]
    res[0] += ao_squared[n2 - 1]
    return res


def lift(a: list[float]) -> list[float]:
    """
    Eleva un polinomio de grado n/2 a uno de grado n, aplicando a(x²).
    Args:
        a : list[float]
            Polinomio en Q[x] / (xⁿ/² + 1).

    Returns:
        list[float]
    """
    n = len(a)
    res = [0] * (2 * n)
    for i in range(n):
        res[2 * i] = a[i]
    return res


def bitsize(a: int) -> int:
    """
    Calcula el tamaño en bits de un número (ignorando el signo), redondeado a múltiplos de 8.

    Esto acelera los cálculos para estimaciones de precisión.

    Args:
        a : int
            Número entero.

    Returns:
        int
            Tamaño en bits redondeado.
    """
    val = abs(a)
    res = 0
    while val:
        res += 8
        val >>= 8
    return res


def reduce(f: list[int], g: list[int], F: list[int], G: list[int]) -> tuple[list[int], list[int]]:
    """
    Reduce el par (F, G) con respecto a (f, g) usando el método de Babai.
    (F, G) <-- (F, G) - k * (f, g), where k = round((F f* + G g*) / (f f* + g g*)).
    Args:
        f, g : list[int]
            Polinomios base del NTRU.
        F, G : list[int]
            Polinomios objetivo a reducir.

    Returns:
        tuple[list[int], list[int]]
            Par reducido (F, G).
    """
    n = len(f)
    size = max(53, bitsize(min(f)), bitsize(max(f)), bitsize(min(g)), bitsize(max(g)))

    f_adjust = [elt >> (size - 53) for elt in f]
    g_adjust = [elt >> (size - 53) for elt in g]
    fa_fft = FFT.fft(f_adjust)
    ga_fft = FFT.fft(g_adjust)

    while 1:
        # Because we work in finite precision to reduce very large polynomials,
        # we may need to perform the reduction several times.
        Size = max(
            53, bitsize(min(F)), bitsize(max(F)), bitsize(min(G)), bitsize(max(G))
        )
        if Size < size:
            break

        F_adjust = [elt >> (Size - 53) for elt in F]
        G_adjust = [elt >> (Size - 53) for elt in G]
        Fa_fft = FFT.fft(F_adjust)
        Ga_fft = FFT.fft(G_adjust)

        den_fft = FFT.add_fft(
            FFT.mul_fft(fa_fft, FFT.adj_fft(fa_fft)), FFT.mul_fft(ga_fft, FFT.adj_fft(ga_fft))
        )
        num_fft = FFT.add_fft(
            FFT.mul_fft(Fa_fft, FFT.adj_fft(fa_fft)), FFT.mul_fft(Ga_fft, FFT.adj_fft(ga_fft))
        )
        k_fft = FFT.div_fft(num_fft, den_fft)
        k = FFT.ifft(k_fft)
        k = [int(round(elt)) for elt in k]
        if all(elt == 0 for elt in k):
            break
        # The two next lines are the costliest operations in ntru_gen
        # (more than 75% of the total cost in dimension n = 1024).
        # There are at least two ways to make them faster:
        # - replace Karatsuba with Toom-Cook
        # - mutualized Karatsuba, see ia.cr/2020/268
        # For simplicity reasons, we didn't implement these optimisations here.
        fk = karamul(f, k)
        gk = karamul(g, k)
        for i in range(n):
            F[i] -= fk[i] << (Size - size)
            G[i] -= gk[i] << (Size - size)
    return F, G


def xgcd(b: int, n: int) -> tuple[int, int, int]:
    """
    Algoritmo extendido de Euclides para enteros.

    Devuelve d, u, v tal que d = u·b + v·n, siendo d el MCD de b y n.

    Args:
        b : int
        n : int

    Returns:
        tuple[int, int, int]
            (d, u, v) tal que d = MCD(b, n) y d = u·b + v·n.
    """
    x0, x1, y0, y1 = 1, 0, 0, 1
    while n != 0:
        q, b, n = b // n, n, b % n
        x0, x1 = x1, x0 - q * x1
        y0, y1 = y1, y0 - q * y1
    return b, x0, y0


def ntru_solve(f: list[int], g: list[int]) -> tuple[list[int], list[int]]:
    """
    Resuelve la ecuación NTRU para los polinomios f y g.

    Devuelve (F, G) tal que f·G - g·F ≡ q mod (xⁿ + 1).
    Args:
        f : list[int]
        g : list[int]
        Formato: coeficiente

    Returns:
        tuple[list[int], list[int]]
            Solución (F, G) del sistema NTRU.
    """
    n = len(f)

    if n == 1:
        f0 = f[0]
        g0 = g[0]
        d, u, v = xgcd(f0, g0)
        if d != 1:
            raise ValueError
        else:
            return [-q * v], [q * u]
    else:
        fp = field_norm(f)
        gp = field_norm(g)
        Fp, Gp = ntru_solve(fp, gp)
        F = karamul(lift(Fp), galois_conjugate(g))
        G = karamul(lift(Gp), galois_conjugate(f))
        F, G = reduce(f, g, F, G)
        return F, G


def isInvertible2(x: list[int]) -> bool:
    """
    Verifica si un polinomio es invertible módulo 2.

    Args:
        x : list[int]
            Polinomio binario.

    Returns:
        bool
            True si la suma de coeficientes es impar (=> invertible), False en caso contrario.
    """
    return (sum(x) % 2) == 1


### Codificación y decodificación



In [ ]:
def decode_private(logn: int, priv: np.ndarray) -> tuple[np.ndarray, list[int], list[int], np.ndarray]:
    """
    Decodifica una clave privada comprimida.(Alg. 5)

    Args:
        logn : int
            Logaritmo base 2 del grado del polinomio (n = 2^logn).
        priv : np.ndarray
            Clave privada codificada como array de bytes.

    Returns:
        tuple[np.ndarray, list[int], list[int], np.ndarray]
            - kgseed: semilla para regenerar (f, g)
            - Fmod2: polinomio F reducido módulo 2
            - Gmod2: polinomio G reducido módulo 2
            - hpub: hash de la clave pública
    """
    n = 1 << logn
    params = HAWKParams.get
    lenkgseed = params(logn, "lenkgseed")
    kgseed = priv[:lenkgseed]
    Fmod2 = PolProp.bytes_to_poly(priv[lenkgseed : lenkgseed + n // 8], 1 << logn)
    Gmod2 = PolProp.bytes_to_poly(priv[lenkgseed + n // 8 : lenkgseed + n // 4], 1 << logn)
    hpub = priv[-params(logn, "lenhpub") :]
    return kgseed, Fmod2, Gmod2, hpub


def encode_private(kgseed: np.ndarray, F: list[int], G: list[int], hpub: np.ndarray) -> np.ndarray:
    """
    Codifica una clave privada a partir de sus componentes.(Alg. 4)

    Args:
        kgseed : np.ndarray
            Semilla para regenerar (f, g).
        F : list[int]
            Polinomio F.
        G : list[int]
            Polinomio G.
        hpub : np.ndarray
            Hash de la clave pública.

    Returns:
        np.ndarray
            Clave privada codificada como array de bytes.
    """

    Fmod2 = [x % 2 for x in F]
    Gmod2 = [x % 2 for x in G]
    return np.append(
        np.append(
            kgseed, np.packbits(np.array(Fmod2 + Gmod2, dtype=bool), bitorder="little")
        ),
        hpub,
    )


def encodeint(x: int, k: int) -> np.ndarray:
    """
    Codifica un entero en binario (Section 3.3.1))

    Args:
        x : int
            Entero no negativo menor que 2^k.
        k : int
            Número de bits a usar en la codificación.

    Returns:
        np.ndarray
            Array de bits, con el bit menos significativo en la posición 0.
    """
    b = np.array([], dtype=bool)
    assert k >= 0
    if k == 0:
        return b  # return an empty sequence
    assert x >= 0
    assert x < 2**k
    n = x
    for _ in range(int(k)):
        b = np.append(b, int(n % 2 == 1))
        n //= 2
    return b


def decodeint(x: np.ndarray | list[int]) -> int:
    """
    Decodifica una secuencia de bits (LSB primero) a un entero.

    Args:
        x : np.ndarray | list[int]
    Returns:
        int
            Valor entero resultante.
    """
    x = np.array(x, dtype=np.uint8)  # Asegura tipo seguro
    assert np.all((x == 0) | (x == 1)), "All elements must be 0 or 1"

    c = 0
    for bit in reversed(x):
        c = (c << 1) | int(bit)
    return int(c)



def compressgr(x: list[int], low: int, high: int) -> np.ndarray | None:
    """
    Comprime una secuencia de enteros con Golomb-Rice.(Alg. 6)

    Args:
        x : list[int]
            Secuencia de enteros a comprimir.
        low : int
            Número de bits para la parte baja.
        high : int
            Límite de bits totales por valor.

    Returns:
        np.ndarray | None
            Array de bits comprimidos, o None si los valores exceden el límite.
    """
    k = len(x)
    assert k % 8 == 0
    for i in range(k):
        assert x[i] < 2 ** high and x[i] > -(2**high)
    # Set y to an empty sequence of bits.
    y = np.array([], dtype=bool)
    # Set v to an empty sequence of integers.
    v = np.array([], dtype=np.int16)
    # for i = 0 to k − 1 do
    #   s = 1 if x[i] < 0, or 0 if x[i] ≥ 0
    #   y = y ∥ s
    #   v = v ∥ x[i] − s(2x[i] + 1)
    #   if v[i] ≥ 2^high then
    #     return ⊥
    for i in range(0, k):
        s = int(x[i] < 0)
        y = np.append(y, bool(s))
        v = np.append(v, x[i] - s * (2 * x[i] + 1))
        if v[i] >= (2**high):
            return None
    # for i = 0 to k − 1 do
    #   y ← y ∥ encodeint(v[i] mod 2^low,low)
    for i in range(k):
        y = np.append(y, encodeint(v[i] % (2**low), low))
    #   for i = 0 to k − 1 do
    #     y ← y ∥ encodeint(0, ⌊v[i]/2^low⌋) ∥ 1
    for i in range(k):
        y = np.append(y, encodeint(0, v[i] // 2**low))
        y = np.append(y, 1)

    return y


def decompressgr(y: np.ndarray, k: int, low: int, high: int) -> tuple[np.ndarray, int] | None:
    """
    Descomprime una secuencia codificada con Golomb-Rice.(Alg. 7)

    Args:
        y : np.ndarray
            Secuencia de bits comprimida.
        k : int
            Número de elementos originales.
        low : int
            Tamaño de la parte baja.
        high : int
            Límite total de bits por valor.

    Returns:
        tuple[np.ndarray, int] | None
            Tupla con los valores descomprimidos y la cantidad de bits usados, o None si falla.
    """
    assert k % 8 == 0
    # if lenbits(y) < k(low + 2) then
    #   return ⊥
    if len(y) < k * (low + 2):
        return None
    # for i = 0 to k − 1 do
    #   x[i] ← decodeint(y[i ·low + k : (i + 1) ·low + k])
    #x = [0] * k
    x = np.zeros(k, dtype=np.int32)
    for i in range(k):
        x[i] = decodeint(y[i * low + k : (i + 1) * low + k])
    # j ← k(low + 1)
    # for i = 0 to k − 1 do
    #   z ← −1
    #   repeat
    #     z ← z + 1
    #     if j ≥ len_bits(y) or z ≥ 2^(high−low) then
    #       return ⊥
    #     t ← y[j]
    #     j ← j + 1
    #   until t = 1
    #   x[i] ← x[i] + z · 2^low
    j = k * (low + 1)
    for i in range(k):
        z = -1
        t = 0
        while t != 1:
            z = z + 1
            if j >= len(y) and z >= 2 ** (high - low):
                return None
            t = y[j]
            j = j + 1
        x[i] = x[i] + z * 2**low
    # for i = 0 to k − 1 do
    #   x[i] ← x[i] − y[i](2x[i] + 1)  ▷ Application of the sign bit.
    for i in range(k):
        x[i] = x[i] - y[i] * (2 * x[i] + 1)
    return (x, j)


def encode_public(logn: int, q00: np.ndarray, q01: np.ndarray) -> np.ndarray | None:
    """
    Codifica una clave pública con compresión Golomb-Rice.(Alg. 8)

    Args:
        logn : int
            log_2 del grado del polinomio.
        q00 : np.ndarray
            Polinomio q00 (int16).
        q01 : np.ndarray
            Polinomio q01 (int16).

    Returns:
        np.ndarray | None
            Clave pública codificada como array de bytes, o None si falla.
    """
    n = 1 << logn
    params = HAWKParams.get
    if q00[0] < -(2**15) or q00[0] >= 2**15:
        return None, False

    v = 16 - params(logn, "high00")

    qp00 = q00.copy()
    qp00[0] = q00[0] // (2**v)

    y00 = compressgr(
        qp00[0 : int(n / 2)], params(logn, "low00"), params(logn, "high00")
    )

    if y00 is None:
        return None

    y00 = np.append(y00, encodeint(q00[0] % (2**v), v))

    while len(y00) % 8 != 0:
        y00 = np.append(y00, 0)

    y01 = compressgr(q01, params(logn, "low01"), params(logn, "high01"))
    if y01 is None:
        return None

    y = np.append(y00, y01)

    if len(y) > params(logn, "lenpub") * 8:
        return None

    while len(y) < params(logn, "lenpub") * 8:
        y = np.append(y, 0)

    return np.packbits(y, axis=None, bitorder="little")


def decode_public(logn: int, pub: np.ndarray) -> tuple[np.ndarray, np.ndarray] | None:
    """
    Decodifica una clave pública codificada.(Alg. 9)

    Args:
        logn : int
            log_2 del grado del polinomio.
        pub : np.ndarray
            Clave pública codificada.

    Returns:
        tuple[np.ndarray, np.ndarray] | None
            Polinomios q00 y q01 (int16), o None si falla.
    """
    n = 1 << logn
    params = HAWKParams.get
    if len(pub) != params(logn, "lenpub"):
        return None

    v = 16 - params(logn, "high00")
    y = np.unpackbits(pub, bitorder="little")

    r00 = decompressgr(y, n // 2, params(logn, "low00"), params(logn, "high00"))
    if r00 is None:
        return None
    r00, j = r00
    q00 = np.zeros(n, dtype=np.int16)
    q00[: n // 2] = r00

    if len(y) * 8 < j + v:
        return None

    q00[0] = 2**v * q00[0] + decodeint(y[j : j + v])

    j = j + v

    while j % 8 != 0:
        if j >= len(y) or y[j] != 0:
            return None
        j += 1

    q00[n // 2] = 0
    for i in range(n // 2 + 1, n):
        q00[i] = -q00[n - i]

    r01 = decompressgr(y[j:], n, params(logn, "low01"), params(logn, "high01"))
    if r01 is None:
        return None
    r01, jp = r01

    j = j + jp
    q01 = np.array(r01, dtype=np.int16)

    while j < len(y):
        if y[j] != 0:
            return None

        j += 1

    return q00, q01


def encode_sign(logn: int, salt: np.ndarray, s1: np.ndarray) -> np.ndarray | None:
    """
    Codifica una firma digital.(Alg. 10)

    Args:
        logn : int
            log_2 del grado del polinomio.
        salt : np.ndarray
            Sal usada para regenerar h0 y h1.
        s1 : np.ndarray
            Polinomio de la firma (int16).

    Returns:
        np.ndarray | None
            Firma codificada como array de bytes, o None si falla.
    """
    params = HAWKParams.get
    y = compressgr(s1, params(logn, "lows1"), params(logn, "highs1"))
    if y is None:
        return None

    leny = (params(logn, "lensig") - params(logn, "lensalt")) * 8
    if len(y) > leny:
        return None

    while len(y) < leny:
        y = np.append(y, 0)

    return np.append(salt, np.packbits(y, bitorder="little"))


def decode_sign(logn: int, sig: np.ndarray) -> tuple[np.ndarray, np.ndarray] | None:
    """
    Decodifica una firma digital.(Alg. 11)

    Args:
        logn : int
            log₂ del grado del polinomio.
        sig : np.ndarray
            Firma codificada.

    Returns:
        tuple[np.ndarray, np.ndarray] | None
            Tupla con la sal y el polinomio s1, o None si falla.
    """
    params = HAWKParams.get
    n = 1 << logn
    y = np.unpackbits(sig, bitorder="little")
    if len(sig) != params(logn, "lensig"):
        return None

    salt = sig[: params(logn, "lensalt")]

    s1 = decompressgr(
        y[params(logn, "lensalt") * 8 :],
        n,
        params(logn, "lows1"),
        params(logn, "highs1"),
    )

    if s1 is None:
        return None

    s1, j = s1
    s1 = np.array(s1, dtype=np.int16)

    j += params(logn, "lensalt") * 8
    while j < len(y):
        if y[j] != 0:
            return None
        j += 1

    return salt, s1


### Recuperación de los polinomios f y g

In [ ]:
def regeneratefg(kgseed: bytes, n: int) -> tuple[list[int], list[int]]:
    """
    Regenera determinísticamente los polinomios (f, g) a partir de una semilla,
    usando una versión paralelizada de SHAKE256. (Alg. 12)

    Args:
        kgseed : bytes
            Semilla de generación clave.
        n : int
            Grado de los polinomios f y g.

    Returns:
        tuple[list[int], list[int]]
            Tupla que contiene:
            - f: polinomio f como lista de enteros.
            - g: polinomio g como lista de enteros.
    """
    b = int(n / 64)
    assert b == 4 or b == 8 or b == 16

    y = SHAKE256x4(kgseed, 2 * n * b // 64)  # array of 64-bit values

    # map y to a sequence of bits
    ybits = [None] * b * 2 * n
    for j, y in enumerate(y):
        for bi in range(64):
            ybits[j * 64 + bi] = (y >> bi) & 1

    f = [int(0)] * n
    for i in range(n):
        sum = 0
        for j in range(b):
            sum += ybits[i * b + j]
        f[i] = sum - b // 2

    g = [0] * n
    for i in range(n):
        sum = 0
        for j in range(b):
            sum += ybits[(i + n) * b + j]
        g[i] = sum - b // 2

    return (f, g)


### Componentes adicionales para la firma

In [ ]:
def samplersign(s: bytes, t: list[int], n: int, T0: list[int], T1: list[int]) -> list[int]:
    """

    Realiza una muestra gaussiana discreta centrada en un vector binario t, utilizando tablas CDT.(Alg. 14)

    Args:
        s : bytes
            Semilla de aleatoriedad.
        t : list[int]
            Vector binario central que determina la paridad (0 o 1).
        n : int
            Grado de los polinomios (debe ser divisible por 8).
        T0 : list[int]
            Tabla CDT para paridad 0.
        T1 : list[int]
            Tabla CDT para paridad 1.

    Returns:
        list[int]
            Polinomio con muestras de la gaussiana discreta.
    """
    y = SHAKE256x4(s, int(5 * n / 2))
    d = [None] * 2 * n

    for j in range(4):
        for i in range(int(n / 8)):
            for k in range(3 + 1):
                r = 16 * i + 4 * j + k
                a = y[j + 4 * (5 * i + k)]
                b = (y[j + 4 * (5 * i + 4)] // (2 ** (16 * k))) % 2**15
                c = (a % 2**63) + 2**63 * b
                (v0, v1) = (0, 0)
                z = 0
                while T0[z] != 0 and T1[z] != 0:
                    if c < T0[z]:
                        v0 = v0 + 1
                    if c < T1[z]:
                        v1 = v1 + 1
                    z = z + 1
                if t[r] == 0:
                    v = 2 * v0
                else:
                    v = 2 * v1 + 1
                if a >= 2**63:
                    v = -v
                d[r] = v
    return d


def symbreak(w: list[int]) -> bool:
    """

    Rompe la simetría en firmas gaussianas: devuelve `True` si el primer coeficiente no nulo de `w`
    es positivo, y `False` si es negativo o todos son cero.(Section 3.5.2)

    Args:
        w : list[int]
            Polinomio sobre el que aplicar el criterio de ruptura de simetría.

    Returns:
        bool
            True si el primer coeficiente no nulo es positivo, False en otro caso.
    """

    for x in w:
        if x != 0:
            if x > 0:
                return True
            else:
                return False
    return False



### Componentes adicionales para la verificación

In [ ]:
def rebuilds0(logn: int, q00: list[int], q01: list[int], w1: list[int], h0: list[int]) -> list[int]:
    """

    Reconstruye el polinomio s0 a partir de la clave pública y los polinomios de la firma.(Alg. 18)

    Args:
        logn : int
            log2 del grado del polinomio (n = 2^logn).
        q00 : list[int]
            Polinomio público q00 (de la clave pública).
        q01 : list[int]
            Polinomio público q01 (de la clave pública).
        w1 : list[int]
            Polinomio w1 (parte de la firma).
        h0 : list[int]
            Polinomio h0 derivado del hash (parte de la firma).

    Returns:
        w0 : list[int]
            Polinomio reconstruido s0.
    """
    #print("q00:", q00, "long:",len(q00))
    #print("q01", q01, "long:",len(q01))
    #print("w1:", w1, "long:", len(w1))
    #print("h0:", h0, "long:", len(h0))
    params = HAWKParams.get
    n = 1 << logn
    cw1 = 2 ** (29 - (1 + params(logn, "highs1")))
    cq00 = 2 ** (29 - params(logn, "high00"))
    cq01 = 2 ** (29 - params(logn, "high01"))
    cs0 = (2 * cw1 * cq01) / (n * cq00)
    w1scalled = (np.array(w1, dtype=np.int32)) * cw1
    w1fft = FFTfp.fft(w1scalled)
    z00 = q00.copy()
    if z00[0] < 0:
        return False
    z00[0] = 0
    q00fft = FFTfp.fft(np.array(z00, dtype=np.int32) * cq00)
    q01fft = FFTfp.fft(np.array(q01, dtype=np.int32) * cq01)
    alpha = (2 * cq00 * np.int64(q00[0])) // n
    for u in range(0, n // 2):
        x_re = np.int64(q01fft[u]) * np.int64(w1fft[u])
        x_re -= np.int64(q01fft[u + n // 2]) * np.int64(w1fft[u + n // 2])

        x_im = np.int64(q01fft[u]) * np.int64(w1fft[u + n // 2])
        x_im += np.int64(q01fft[u + n // 2]) * np.int64(w1fft[u])

        (x_re, z_re) = (np.abs(x_re), FFTfp.sgn(x_re))
        (x_im, z_im) = (np.abs(x_im), FFTfp.sgn(x_im))

        v = alpha + q00fft[u]
        if v <= 0 or v >= 2**32 or x_re >= v * 2**32 or x_im >= v * 2**32:
            return False

        y_re = np.int32(x_re // v)
        y_im = np.int32(x_im // v)

        q01fft[u] = y_re - 2 * z_re * y_re
        q01fft[u + n // 2] = y_im - 2 * z_im * y_im

    t = FFTfp.invfft(q01fft)
    w0 = np.zeros(n, dtype=np.int32)
    for u in range(n):
        v = cs0 * np.int32(h0[u]) + np.int32(t[u])
        z = (v + cs0) // (2 * cs0)
        if z < -(2 ** params(logn, "highs0")) or z >= 2 ** (params(logn, "highs0")):
            return False
        w0[u] = np.int32(h0[u]) - 2 * z

    return w0
def polyQnorm(q00: list[int], q01: list[int], w0: list[int], w1: list[int], p: int) -> int:
    """

    Calcula la norma Q de un par de polinomios (w0, w1) utilizando la métrica definida por la clave pública.(Alg. 19)

    Args:
        q00 : list[int]
            Polinomio de la clave pública que representa ⟨q00⟩.
        q01 : list[int]
            Polinomio de la clave pública que representa ⟨q01⟩.
        w0 : list[int]
            Parte del vector de firma.
        w1 : list[int]
            Parte del vector de firma.
        p : int
            Primo usado para los cálculos módulo p.

    Returns:
        int
            Valor de ||W||²_Q ≡ ⟨W, Q·W⟩ mod p.
    """
    zetas, _ = NTT.get_roots(p, len(q00))
    q00ntt = NTT.ntt(q00, p, zetas)
    q01ntt = NTT.ntt(q01, p, zetas)
    w0ntt = NTT.ntt(w0, p, zetas)
    w1ntt = NTT.ntt(w1, p, zetas)

    d = [(w1 * pow(int(q00), p - 2, p)) % p for (w1, q00) in zip(w1ntt, q00ntt)]
    c = [(d * w1adj) % p for (d, w1adj) in zip(d, NTT.ntt(PolOp.adjoint(w1), p))]
    acc = sum(c)
    e = [(int(w0) + d * int(q01)) % p for (w0, d, q01) in zip(w0ntt, d, q01ntt)]
    c = [
        (int(q00) * int(e) * int(eadj)) % p
        for (q00, e, eadj) in zip(q00ntt, e, NTT.nttadj(e, p))
    ]
    acc += sum(c)
    return acc % p


## Algoritmos finales

### Algoritmo de Generación

In [ ]:
def hawkkeygen(logn: int, rng: RngContext = None) -> tuple[np.ndarray, np.ndarray]:
    """

    Genera un par de claves pública y privada codificadas, utilizando generación determinista de (f, g).(Alg. 13)

    Args:
        logn : int
            log_2 del grado de los polinomios.
        rng : RngContext, opcional
            Generador de números aleatorios determinista. Si no se proporciona, se genera uno con entropía aleatoria.

    Returns:
        tuple[np.ndarray, np.ndarray]
            Par (priv, pub) con las claves codificadas:
                - priv : clave privada (uint8)
                - pub : clave pública (uint8)
    """

    if rng is None:
        rng = RngContext(np.random.randint(0, 256, 40, dtype=np.uint8))

    _, _, _, _, _, _, _, priv, pub = hawkkeygen_unpacked(logn, rng)
    return priv, pub


def hawkkeygen_unpacked(logn: int, rng: RngContext = None) -> tuple[list[int], list[int], list[int], list[int], list[int], list[int], np.ndarray, np.ndarray, np.ndarray]:
    """

    Variante extendida de hawkkeygen que expone todos los polinomios intermedios generados durante la creación de la clave.(Alg. 13)
    Se utiliza para pruebas, depuración y generación detallada.

    Args:
        logn : int
            log_2 del grado de los polinomios.
        rng : RngContext, opcional
            Contexto RNG determinista para generación de semilla kgseed. Si no se proporciona, se usa uno aleatorio.

    Returns:
        tuple[
            list[int], list[int], list[int], list[int], list[int], list[int],
            np.ndarray, np.ndarray, np.ndarray
        ]
            Tupla con:
                - f : polinomio secreto f (int8)
                - g : polinomio secreto g (int8)
                - F : polinomio secreto F (int8)
                - G : polinomio secreto G (int8)
                - q00 : polinomio público q00 (int16)
                - q01 : polinomio público q01 (int16)
                - kgseed : semilla para regenerar (f, g) (uint8)
                - priv : clave privada codificada (uint8)
                - pub : clave pública codificada (uint8)
    """
    n = 1 << logn
    params = HAWKParams.get
    if rng is None:
        rng = RngContext(np.random.randint(0, 256, 40, dtype=np.uint8))

    # Line 1: kgseen <- Rnd(len_bits(kgseed))
    kgseed = rng.random(params(logn, "lenkgseed"))

    # Line 2: (f, g) <- Regeneratefg(kgseed)
    f, g = regeneratefg(kgseed.tobytes(), n)

    # Line 3: if isInvertible(f, 2) false or isInvertible(f, 2) is false then
    if not PolProp.isinvertible(f, 2) or not PolProp.isinvertible(g, 2):
        # Line 4: restart
        return hawkkeygen_unpacked(logn, rng)

    fadj = PolOp.adjoint(f)
    gadj = PolOp.adjoint(g)

    # Line 5: if ||(f,g)||2 <= 2*n*sigkrsec**2:
    if (PolProp.l2norm(f) + PolProp.l2norm(g)) <= (2 * n * (params(logn, "sigmakrsec") ** 2)):
        return hawkkeygen_unpacked(logn, rng)

    p = (1 << 16) + 1

    # Line 7: q00 <- f f* + g g*
    q00 = PolOp.poly_add(NTT.poly_mul_ntt(f, fadj, p), NTT.poly_mul_ntt(g, gadj, p))

    # Line 8: (p1, p2) <- (2147473409, 2147389441)
    p1, p2 = (2147473409, 2147389441)

    # Line 9: if isInvertible(q00, p1) false or isInvertible(q00, p2) is false then
    if not PolProp.isinvertible(q00, p1) or not PolProp.isinvertible(q00, p2):
        # Line 10: restart
        return hawkkeygen_unpacked(logn, rng)

    # Line 11: if (1/q00)[0] >= beta0 then
    invq00 = FFT.inv_fft(q00)
    if invq00[0] >= params(logn, "beta0"):
        # Line 12: restart
        return hawkkeygen_unpacked(logn, rng)

    try:
        # Line 13: r <- NTRUSolve(f, g, 1)
        # Line 16: (F, G) <- r
        F, G = ntru_solve(f, g)
    except ValueError:
        # Line 14&15: if r = \bot then restart
        return hawkkeygen_unpacked(logn, rng)

    # Line 17: if infnorm( (F,G) ) > 127 then
    if PolProp.infnorm(F) > 127 or PolProp.infnorm(G) > 127:
        # Line 18: restart
        return hawkkeygen_unpacked(logn, rng)

    Fadj = PolOp.adjoint(F)
    Gadj = PolOp.adjoint(G)

    # Line 19: q01 <- F f* + G g*
    q01 = PolOp.poly_add(NTT.poly_mul_ntt(F, fadj, p), NTT.poly_mul_ntt(G, gadj, p))
    p = 8380417
    # Line 20: q11 <- F F* + G G*
    q11 = PolOp.poly_add(NTT.poly_mul_ntt(F, Fadj, p), NTT.poly_mul_ntt(G, Gadj, p))

    # Line 21: if |q11[i]| >= 2^high11 for any i > 0 then
    if any(abs(q11i) >= 2 ** (params(logn, "high11")) for q11i in q11[1:]):
        # Line 22: restart
        return hawkkeygen_unpacked(logn, rng)

    # Line 23: pub <- EncodePublic(q00, q01)
    pub = encode_public(logn, q00, q01)

    # Line 24: if pub = \bot then
    if pub is None:
        # Line 25: restart
        return hawkkeygen_unpacked(logn, rng)

    # Line 16: hpub <- SHAKE256(pub)
    shake256 = hashlib.shake_256()
    shake256.update(pub.tobytes())
    hpub = np.frombuffer(shake256.digest(params(logn, "lenhpub")), dtype=np.uint8)

    # Line 27: priv <- EncodePrivate(kgseen, F mod 2, G mod 2, hpub)
    priv = encode_private(kgseed, F, G, hpub)

    # Line 28: return (priv, pub)
    return f, g, F, G, q00, q01, kgseed, priv, pub

In [ ]:
logn = 8
priv, pub = hawkkeygen(logn)
print("Clave privada (codificada):", priv[0:10], "longitud:", len(priv))
print("Clave pública (codificada):", pub[0:10], "longitud:", len(pub))


Clave privada (codificada): [ 75  72 173  82  40 226 106 144 209  13] longitud: 96
Clave pública (codificada): [ 30 123 225 158 114 115 161 216  67 237] longitud: 450


### Algoritmo de firma

In [ ]:
def hawksign(logn: int, priv: np.ndarray, msg: np.ndarray, rng: RngContext = None) -> np.ndarray:
    """

    Genera una firma digital válida de un mensaje dado, usando la clave privada codificada.(Alg. 15)

    Args:
        logn : int
            log_2 del grado del polinomio.
        priv : np.ndarray
            Clave privada codificada (uint8).
        msg : np.ndarray
            Mensaje que se desea firmar.
        rng : RngContext, opcional
            Generador de números aleatorios determinista (si no se proporciona, se crea uno aleatorio).

    Returns:
        np.ndarray
            Firma codificada (uint8) del mensaje bajo la clave privada.
    """

    # Line 1: (kgseed, F mod 2, G mod 2, hpub) <- DecodePrivate(priv)
    kgseed, Fmod2, Gmod2, hpub = decode_private(logn, priv)

    while True:
        salt, s1 = hawksign_unpacked(logn, Fmod2, Gmod2, kgseed, hpub, msg, rng)

        # Line 18: EncodeSignature(salt, s1)
        sig = encode_sign(logn, salt, s1)

        # Line 19: if sig != bottom then
        if sig is not None:
            # Line 20: return sig
            return sig


def hawksign_unpacked(
    logn: int,
    Fmod2: list[int],
    Gmod2: list[int],
    kgseed: np.ndarray,
    hpub: np.ndarray,
    msg: np.ndarray,
    rng: RngContext = None
) -> tuple[np.ndarray, list[int]]:
    """

    Subrutina principal de generación de firma con claves privadas ya decodificadas. Produce la sal y el polinomio s_1.(Alg. 15)

    Args:
        logn : int
            log₂ del grado del polinomio.
        Fmod2 : list[int]
            Polinomio F reducido módulo 2.
        Gmod2 : list[int]
            Polinomio G reducido módulo 2.
        kgseed : np.ndarray
            Semilla de generación determinista de (f, g).
        hpub : np.ndarray
            Hash de la clave pública.
        msg : np.ndarray
            Mensaje a firmar.
        rng : RngContext, opcional
            Generador de números aleatorios determinista.

    Returns:
        tuple[np.ndarray, list[int]]
            - salt : np.ndarray
                Sal aleatoria usada para derivar h₀ y h₁.
            - s1 : list[int]
                Polinomio de firma.
    """

    params = HAWKParams.get
    if rng is None:
        rng = RngContext(np.random.randint(0, 256, 32))

    n = 1 << logn

    # Line 2: (f,g) <- Regenerate(kgseed)
    f, g = regeneratefg(kgseed.tobytes(), n)

    # Line 3: M <- SHAKE256(m || hpub)[0:512]
    shake256 = hashlib.shake_256()
    shake256.update(msg.tobytes() + hpub.tobytes())
    M = shake256.digest(64)

    # Line 4: a <- 0
    a = 0

    # prime used for NTTs
    p = (1 << 16) + 1

    # Line 5: loop
    while True:
        # Line 6: salt <- SHAKE256(M||kgseed||EncodeInt(a,32)||Rnd(saltlen))[0:saltlenbits]
        shake256 = hashlib.shake_256()
        shake256.update(
            M
            + kgseed.tobytes()
            + a.to_bytes(4, "little")
            + rng.random(params(logn, "lensalt")).tobytes()
        )
        salt = np.frombuffer(shake256.digest(params(logn, "lensalt")), dtype=np.uint8)

        # Line 7: (h0,h1) <- SHAKE256(M||salt)[0:2n]
        shake256 = hashlib.shake_256()
        shake256.update(M + salt.tobytes())

        h = shake256.digest(n // 4)
        h0 = PolProp.bytes_to_poly(h[: n // 8], n)
        h1 = PolProp.bytes_to_poly(h[n // 8 :], n)

        # Line 8: (t0,t1) <- ((h0*f + f1*F) mod 2, (h0*g + f1*G) mod 2)
        t0 = [
            x % 2 for x in PolOp.poly_add(NTT.poly_mul_ntt(h0, f, p), NTT.poly_mul_ntt(h1, Fmod2, p))
        ]
        t1 = [
            x % 2 for x in PolOp.poly_add(NTT.poly_mul_ntt(h0, g, p), NTT.poly_mul_ntt(h1, Gmod2, p))
        ]

        # Line 9: s <- M || kgseed || EncodeInt(a+1,32) || Rnd(320)
        seed = rng.random(320 // 8)
        s = M + kgseed.tobytes() + (a + 1).to_bytes(4, "little") + seed.tobytes()

        # Line 10: (d0, d1) <- SampleSign(s,(t0,t1))
        d = samplersign(s, t0 + t1, n, params(logn, "T0"), params(logn, "T1"))
        d0 = d[:n]
        d1 = d[n:]

        # Line 11: a <- a + 2
        a = a + 2

        # Line 12: if ||(d0,d1)||2 > 8 * n * sigm_verify^2
        if PolProp.l2norm(d0) + PolProp.l2norm(d1) > 8 * n * (params(logn, "sigmaverify") ** 2):
            # Line 13: continue loop
            continue

        # Line 14: f * d1 - g * d0
        w1 = PolOp.poly_sub(NTT.poly_mul_ntt(f, d1, p), NTT.poly_mul_ntt(g, d0, p))

        # Line 15: if sym-break(w1) ==
        if not symbreak(w1):
            # Line 16: w1 <- -w1
            w1 = [-x for x in w1]

        # Line 17: s <- (h1 - w1) / 2
        sig = [x // 2 for x in PolOp.poly_sub(h1, w1)]

        return salt, sig


In [ ]:
logn = 8
rng = RngContext(np.random.randint(0, 256, 40, dtype=np.uint8))
msg = np.random.randint(0, 256, 128, dtype=np.uint8)
priv, pub = hawkkeygen(logn, rng)
sig = hawksign(logn, priv, msg, rng)
print("Firma generada:", sig[0:10])
print("Tamaño de sig:", len(sig))


Firma generada: [164 156 189  51  58 127 241   8 134 211]
Tamaño de sig: 249


### Algoritmo de verificación.

In [ ]:
def hawkverify(logn: int, pub: np.ndarray, msg: np.ndarray, sig: np.ndarray) -> bool:
    """

    Verifica si una firma es válida para un mensaje dado bajo la clave pública.(Alg. 20)

    Args:
        logn : int
            log_2 del grado del polinomio.
        pub : np.ndarray
            Clave pública codificada (uint8).
        msg : np.ndarray
            Mensaje firmado (uint8).
        sig : np.ndarray
            Firma candidata (uint8).

    Returns:
        bool
            True si la firma es válida, False en caso contrario.
    """
    params = HAWKParams.get
    n = 1 << logn

	# Line 1: r <- DecodeSignature(sig)
    r = decode_sign(logn, sig)
	# Line 2: if r == ⊥ then
    if r is None:
		#Line 3: return False
        return False
	# Line 4: (salt,s1) <- r
    salt, s1 = r

	# Line 5: DecodePublic(pub)
    r = decode_public(logn, pub)
	# Line 6: if r == ⊥ then
    if r is None:
		#Line 7: return False
        return False
	# Line 8: (q00,q01) <- r
    q00, q01 = r

	# Line 9: hpub <- SHAKE256(pub)
    shake256 = hashlib.shake_256()
    shake256.update(pub.tobytes())
    hpub = shake256.digest(params(logn, "lenhpub"))

	# Line 10: M <- SHAKE256( m || hpub)
    shake256 = hashlib.shake_256()
    shake256.update(msg.tobytes() + hpub)
    M = shake256.digest(64)

	# Line 11: (h0,h1) <- SHAKE256(M||salt)[0:2n]
    shake256 = hashlib.shake_256()
    shake256.update(M + salt.tobytes())
    h = shake256.digest(n // 4)
    h0 = PolProp.bytes_to_poly(h[: n // 8], n)
    h1 = PolProp.bytes_to_poly(h[n // 8 :], n)

    f = hawkverify_unpacked(logn, s1, q00, q01, h0, h1)
    return f


def hawkverify_unpacked(
    logn: int,
    s1: list[int],
    q00: np.ndarray,
    q01: np.ndarray,
    h0: list[int],
    h1: list[int]
) -> bool:
    """

    Subrutina de verificación de firma para HAWK. Usa directamente polinomios ya decodificados.(Alg. 20)

    Args:
        logn : int
            log_2 del grado del polinomio.
        s1 : list[int]
            Polinomio de la firma.
        q00 : np.ndarray
            Polinomio público q00 (int16).
        q01 : np.ndarray
            Polinomio público q01 (int16).
        h0 : list[int]
            Polinomio h0 (derivado de hash).
        h1 : list[int]
            Polinomio h1 (derivado de hash).

    Returns:
        bool
            True si la firma es válida, False si no pasa alguna condición de verificación.
    """
    params = HAWKParams.get
    n = 1 << logn

	# Line 12: w1 <- h1 - 2 * s1
    w1 = PolOp.poly_sub(h1, 2 * np.array(s1))

	# Line 13: if sym-break(w1) == false then
    if symbreak(w1) == False:
		# Line 14: Return False
        return False

	# Line 15: w0 <- RebuildS0(q00,q01,w1,h0)
    w0 = rebuilds0(logn, q00, q01, w1, h0)
	# Line 16: if w0 == ⊥ then
    if w0 is None:
		# Line 17: return False
        return False

    p1, p2 = (2147473409, 2147389441)

	# Line 18: r1 <- PolyQnorm(q00,q01,w0,w1,p1)
    r1 = polyQnorm(q00, q01, w0, w1, p1)
	# Line 19: r2 <- PolyQnorm(q00,q01,w0,w1,p2)
    r2 = polyQnorm(q00, q01, w0, w1, p2)

	# Line 20: if r1 != r2 or r1 != 0 mod n then
    if r1 != r2 or r1 % n != 0:
		# Line 21: return False
        return False

	# Line 22 : r1 <- r1 / n
    r1 = r1 // n

	# Line 23: if r1 > 8 * n * sigmaverfy**2 then
    if r1 > 8 * n * params(logn, "sigmaverify") ** 2:
		# Line 24: return False
        return False

	# Line 25: return True
    return True



In [ ]:
# Verificar
result = hawkverify(logn, pub, msg, sig)
print("¿Firma verificada?", result)

¿Firma verificada? True


## Prueba final

In [ ]:
print(f"\n== Prueba con logn = {logn} ==")
rng = RngContext(np.random.randint(0, 256, 40, dtype=np.uint8))
msg = np.random.randint(0, 256, 128, dtype=np.uint8)

# Generar claves con implementación Python
priv, pub = hawkkeygen(logn, rng)
print("Clave privada generada (bytes):", priv," longitud:", len(priv))
print("Clave pública generada (bytes):", pub," longitud:", len(pub))

# Firmar el mensaje
sig = hawksign(logn, priv, msg, rng)
print("Firma generada:", sig)
print("Tamaño de sig:", len(sig))

# Verificar
result = hawkverify(logn, pub, msg, sig)
print("¿Firma verificada?", result)


## Experimento

In [ ]:
def instancies(n : int, logn : int) -> tuple[int]:
    t = 0
    f = 0
    for i in range(n):
        rng = RngContext(np.random.randint(0, 256, 40, dtype=np.uint8))
        msg = np.random.randint(0, 256, 128, dtype=np.uint8)
        priv,pub = hawkkeygen(logn,rng)
        sig = hawksign(logn, priv, msg, rng)
        result = hawkverify(logn, pub, msg, sig)
        if result:
            t += 1
            continue
        f += 1
    return t,f

In [ ]:
iteraciones = 100
log = 8
t,f= instancies(iteraciones, log)
print(f"Se han completado {iteraciones} iteraciones y con {f} fallos")

Se han completado 100 iteraciones y con 0 fallos


In [ ]:
import time
def instanciesv2(n : int, logn : int) -> tuple[int, int, float, float]:
    t = 0
    f = 0
    total = 0.0
    for i in range(n):
        start = time.perf_counter()
        rng = RngContext(np.random.randint(0, 256, 40, dtype=np.uint8))
        msg = np.random.randint(0, 256, 128, dtype=np.uint8)
        priv,pub = hawkkeygen(logn,rng)
        sig = hawksign(logn, priv, msg, rng)
        result = hawkverify(logn, pub, msg, sig)
        end = time.perf_counter()
        total += end - start
        if result:
            t += 1
            continue
        f += 1
    mean = total / n
    return t, f, mean, total

In [ ]:
iteraciones = 20
log = 8
t, f, mean, total = instanciesv2(iteraciones, log)
print(f"Se han completado {iteraciones} iteraciones y con {f} fallos")
print(f"Tiempo promedio de ejecución: {mean} segundos")
print(f"Tiempo total de ejecución: {total} segundos")

Se han completado 20 iteraciones y con 0 fallos
Tiempo promedio de ejecución: 18.97488706394979 segundos
Tiempo total de ejecución: 379.4977412789958 segundos


## Algoritmo High Level

### Funciones auxiliares

In [ ]:
logn = 8
params = HAWKParams.get
kgseed = rng.random(params(logn, "lenkgseed"))
n = 1 << logn
def pol_sample():
    rng = RngContext(np.random.randint(0, 256, 40, dtype=np.uint8))
    kgseed = rng.random(params(logn, "lenkgseed"))
    f, g = regeneratefg(kgseed.tobytes(), n)
    return f, g

def f_g_conditions(f,g):
    if not PolProp.isinvertible(f, 2) or not PolProp.isinvertible(g, 2):
        return False
    fadj = PolOp.adjoint(f)
    gadj = PolOp.adjoint(g)
    # Line 5: if ||(f,g)||2 <= 2*n*sigkrsec**2:
    if (PolProp.l2norm(f) + PolProp.l2norm(g)) <= (2 * n * (params(logn, "sigmakrsec") ** 2)):
        return False
    return True

def Gram(A):
    f = A[0]
    g = A[1]
    F = A[2]
    G = A[3]
    p = (1 << 16) + 1
    fadj = PolOp.adjoint(f)
    gadj = PolOp.adjoint(g)
    q00 = PolOp.poly_add(NTT.poly_mul_ntt(f, fadj, p), NTT.poly_mul_ntt(g, gadj, p))
    q01 = PolOp.poly_add(NTT.poly_mul_ntt(F, fadj, p), NTT.poly_mul_ntt(G, gadj, p))
    q10 = PolOp.adjoint(q01)
    p = 8380417
    Fadj = PolOp.adjoint(F)
    Gadj = PolOp.adjoint(G)
    q11 = PolOp.poly_add(NTT.poly_mul_ntt(F, Fadj, p), NTT.poly_mul_ntt(G, Gadj, p))
    return np.array([q00, q01, q10, q11])

def KGen_encoding(Q,B):
    q11 = Q[3]
    p = 8380417
    if any(abs(q11i) >= 2 ** (params(logn, "high11")) for q11i in q11[1:]):
        # Line 22: restart
        return False
    return True

def H(Q):
  return 0

### Generación de Claves

In [ ]:
def keygen():
  f,g = pol_sample()
  if not f_g_conditions(f,g):
    return keygen()
  try:
    F, G = ntru_solve(f, g)
  except ValueError:
    return keygen()
  B = np.array([f, F, g, G])
  Q = Gram(B)
  if not KGen_encoding(Q,B):
    return keygen()
  return Q,B
keygen()

(array([[ 6079,  1366,  1200, ...,  -245, -1200, -1366],
        [  -62,  1424,   776, ...,   -13,   -69,  -537],
        [  -62,   537,    69, ...,   -14,  -776, -1424],
        [ 5350,   222,   524, ...,  -226,  -524,  -222]]),
 array([[ 1,  0,  0, ..., -2, -1, -1],
        [ 4,  7,  1, ..., -3,  2, -8],
        [ 0,  1,  1, ...,  0, -1,  0],
        [ 2, -1,  2, ...,  0,  1,  2]]))

In [ ]:
def sign(msg,sk,hpub):
